In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, grid_search
from sklearn.dummy import DummyRegressor
from sklearn.cross_validation import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

In [142]:
df = pd.read_excel("data/Data_Puccia_paCelia.xls")
df = df.dropna()
Xs = df[['LAT', 'bio1', 'bio12', 'alt']]
y = df['ni']

minmax_y = MinMaxScaler([-1, 1])
minmax_x = MinMaxScaler([-1, 1])
y = minmax_y.fit_transform(y)
Xs = minmax_x.fit_transform(Xs)

X_train, X_test, y_train, y_test = train_test_split(Xs, y, random_state=42, test_size=0.2)

In [143]:
dummyr = DummyRegressor()
dummyr.fit(X_train, y_train)
y_pred = dummyr.predict(X_test)
print r2_score(y_test, y_pred)

-0.00470745901837


### Epsilon-Support Vector Regression.



The coefficient R^2 is defined as (1 - u/v), where u is the regression sum of squares ((y_true - y_pred) ** 2).sum() and v is the residual sum of squares ((y_true - y_true.mean()) ** 2).sum(). Best possible score is 1.0

In [144]:
svr = svm.SVR(kernel='rbf', C=1e3, gamma=0.009)
y_lin = svr.fit(Xs, y).predict(Xs)
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)
print r2_score(y_test, y_pred)
print len(svr.support_vectors_)

0.663530607593
40


In [ ]:
parameters = {'C': [1e3, 1e5], 'gamma': np.arange(0.0001, 0.01, 0.0001)}
clf = grid_search.GridSearchCV(svr, parameters)
clf.fit(X_train, y_train)
clf.best_estimator_

In [166]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

regr_1 = DecisionTreeRegressor(max_depth=2, max_features='auto')
regr_1.fit(X_train, y_train)
y_pred_tree = regr_1.predict(X_test)
print regr_1.feature_importances_
print r2_score(y_test, y_pred_tree)

[ 0.72722441  0.          0.          0.27277559]
0.577324269073


In [163]:
regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=2),
                          n_estimators=100, random_state=np.random.RandomState(1))
regr_2.fit(X_train, y_train)
print regr_2.feature_importances_
y_pred_ada = regr_2.predict(X_test)
print r2_score(y_test, y_pred_ada)
print regr_2.score(X_test, y_test)

[ 0.38428356  0.13055293  0.19321545  0.29194806]
0.709922936209
0.709922936209


In [136]:
from nolearn.lasagne import NeuralNet
from lasagne.layers import *
from lasagne.updates import nesterov_momentum
from nolearn.lasagne.visualize import plot_loss
from nolearn.lasagne.visualize import plot_conv_weights
from nolearn.lasagne.visualize import plot_conv_activity
from nolearn.lasagne.visualize import plot_occlusion


In [137]:
layers0 = [('input', InputLayer), 
            ('hidden1', DenseLayer), 
            #('dropout1', DropoutLayer), 
            ('hidden2', DenseLayer), 
            #('dropout2', DropoutLayer), 
            ('hidden3', DenseLayer), 
            ('dropout3', DropoutLayer), 
            ('output', DenseLayer)]

net0 = NeuralNet(layers=layers0,
                input_shape=(None, 4),
                hidden1_num_units=200,
                #dropout1_p=0.3,
                hidden2_num_units=500,
                #dropout2_p=0.4,
                hidden3_num_units=200,
                dropout3_p=0.3,
                output_num_units=1,
                output_nonlinearity=None,
                update=nesterov_momentum,
                update_learning_rate=0.005,
                update_momentum=0.9,
                #eval_size=0.1,
                verbose=1,
                regression=True,
                max_epochs=1000)

In [138]:
df = pd.read_excel("data/Data_Puccia_paCelia.xls")
df = df.dropna()
Xs = np.asarray(df[['LAT', 'bio1', 'bio12', 'alt']], dtype=np.float32)
y = np.asarray(df['vin'], dtype=np.float32)

minmax_y = MinMaxScaler([0, 1])
minmax_x = MinMaxScaler([0, 1])
y = minmax_y.fit_transform(y)
Xs = minmax_x.fit_transform(Xs)

X_train, X_test, y_train, y_test = train_test_split(Xs, y, random_state=42, test_size=0.2)

In [139]:
net0.fit(X_train, y_train)

# Neural Network with 201901 learnable parameters

## Layer information

  #  name        size
---  --------  ------
  0  input          4
  1  hidden1      200
  2  hidden2      500
  3  hidden3      200
  4  dropout3     200
  5  output         1

  epoch    train loss    valid loss    train/val  dur
-------  ------------  ------------  -----------  -----
      1       0.33240       0.24202      1.37343  0.00s
      2       0.30402       0.20363      1.49296  0.00s
      3       0.25377       0.16500      1.53801  0.00s
      4       0.21099       0.12897      1.63592  0.00s
      5       0.16636       0.09954      1.67131  0.00s
      6       0.13561       0.07784      1.74229  0.00s
      7       0.10476       0.06349      1.65001  0.01s
      8       0.08783       0.05606      1.56668  0.00s
      9       0.06916       0.05406      1.27917  0.01s
     10       0.06614       0.05597      1.18184  0.00s
     11       0.06594       0.06039      1.09198  0.00s
     12       0.06541   

NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x7f2af9615810>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x7f2af9615790>,
     custom_score=None, dropout3_p=0.3, hidden1_num_units=200,
     hidden2_num_units=500, hidden3_num_units=200, input_shape=(None, 4),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('hidden1', <class 'lasagne.layers.dense.DenseLayer'>), ('hidden2', <class 'lasagne.layers.dense.DenseLayer'>), ('hidden3', <class 'lasagne.layers.dense.DenseLayer'>), ('dropout3', <class 'lasagne.layers.noise.DropoutLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=1000, more_params={},
     objective=<function objective at 0x7f2af9617c08>,
     objective_loss_function=<function squared_error at 0x7f2af960e0c8>,
     on_epoch_finished=[<nolearn.lasagne.handlers.PrintLog instance at 0x7f2aeafc5950>],
     on_training_finished=[],
     on_tr

In [141]:
r2_score(y_test, net0.predict(X_test))

0.20684273996093894

In [127]:
plot_loss(net0)
plt.show()

### Mapas

In [94]:
import pandas as pd
import numpy as np
from matplotlib.mlab import griddata
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.patches as patches
from matplotlib import cm
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from scipy.interpolate import Rbf
import shapefile
import argparse
import sys

In [95]:
class MapData(object):
    """
    On map data you can find the coordinates and the ancestry information
    ordered by code.
    """
    def __init__(self, filename, columns):
        """
        load the data files (coord and ancestry) and merge the data by code.
        """
        super(MapData, self).__init__()
        
        self.df = self.load_file(filename, columns)

    def load_file(self, filename, columns):
        """the file has to be CODE Lat Lon """
        df = pd.read_excel(filename)
        df = df.dropna()
        df_values = df[['LAT', 'LON'] + columns]
        return df_values

    def get_ancestry_average_by_coordinates(self, name_ancestry):
        """
        In this method we take the average of each coordinate in the df
        """
        g = self.df.groupby(['LAT', 'LON'])
        list_coord = list(g)
        self.ancestry_avg = np.array(map(lambda i: np.average(i[1][name_ancestry].values) * 100, list_coord))

    def get_coordinates(self):
        """
        give the coordinates to do the mesh (can't be duplicate data)
        """
        self.coordinates = self.df[['LAT', 'LON']].drop_duplicates()

    def project_coordinates(self, m):
        self.coordinates['projected_lon'], self.coordinates['projected_lat'] = m(*(self.coordinates['LON'].values, self.coordinates['LAT'].values))
        lat = [83.10138, 62.1860, -59.9770, -62.1860]
        lon = [-22.8515, 179.1210, -177.3632, -17.5781]
        self.rect_lon, self.rect_lat = m(*(lon, lat))

    def interpolate(self, numcols=100, numrows=100):
        """
        Take the convex hull of all cordinates to generate a meshgrid
        """
        #xi = np.linspace(self.coordinates['projected_lon'].min(), self.coordinates['projected_lon'].max(), numcols)
        #yi = np.linspace(self.coordinates['projected_lat'].min(), self.coordinates['projected_lat'].max(), numrows)
        xi = np.linspace(min(self.rect_lon), max(self.rect_lon), numcols)
        yi = np.linspace(min(self.rect_lat), max(self.rect_lat), numrows)
        
        xi, yi = np.meshgrid(xi, yi)
        # interpolate
        x, y, z = self.coordinates['projected_lon'].values, self.coordinates['projected_lat'].values, self.df[column].values.ravel()
        interp = Rbf(x, y, z, smooth=0.01, fuction='thin_plate')
        zi = interp(xi, yi)
        zi = np.clip(zi, a_min=0., a_max=100.)
        #zi = griddata(x, y, z, xi, yi)

        return xi, yi, zi, x, y, z

In [155]:
class MainDisplay(object):
    """In this class we have the reference to our display map and the method of how to draw it."""
    def __init__(self, lllon=-180, lllat=-80, urlon=0, urlat=40, figsize=(11.7,8.3), fs='data/continents/continent'):
        super(MainDisplay, self).__init__()
        plt.clf()
        self.fig = plt.figure(figsize=figsize)
        self.ax = self.fig.add_subplot(111, axisbg='w', frame_on=False)
        self.anc_map = Basemap(projection = 'merc', llcrnrlon = lllon,
                                llcrnrlat = lllat, urcrnrlon = urlon,
                                urcrnrlat = urlat, resolution='h')
        self.anc_map.readshapefile(fs, 'borders', drawbounds=False, linewidth=0.8)
    
    def draw(self, xi, yi, zi, x, y, z, coordinates, ancestry, clips):
        """
        This methods display the ancestry data from the MapData class, in this
        method you can setup the color display and resolution of the map.
        """
        norm = Normalize()
        self.anc_map.drawmapboundary(fill_color = 'white')
        self.anc_map.fillcontinents(color='#C0C0C0', lake_color='#7093DB')
        self.anc_map.drawcountries(
            linewidth=.75, linestyle='solid', color='#000073',
            antialiased=True,
            ax=self.ax, zorder=3)

        # contour plot
        con_ = self.anc_map.contour(xi, yi, zi, zorder=6, levels=np.arange(round(min(z)) -1., round(max(z)), 0.5), colors=('k',), linewidths=(0.2,), alpha=0.5)
        con = self.anc_map.contourf(xi, yi, zi, zorder=5, cmap='jet', levels=np.arange(round(min(z)-1.), round(max(z)), 0.05))
        
        #check alpha parameter for areas without data
        # clip the data so only display the data inside of the country
        #for shape_clip in clips:
        for contour in con_.collections:
            contour.set_clip_path(clips)
        
        for contour in con.collections:
            contour.set_clip_path(clips)
        
        # scatter plot
        self.anc_map.scatter(
            coordinates['projected_lon'],
            coordinates['projected_lat'],
            color='#545454',
            edgecolor='#ffffff',
            alpha=.75,
            s=30 * norm(ancestry),
            cmap='RdPu',
            ax=self.ax,
            vmin=zi.min(), vmax=zi.max(), zorder=5)

        # add colour bar
        cbar = self.anc_map.colorbar()

# TODO move this to an other module
def process_shapefile(filename_shp, my_map, ax):
    # http://basemaptutorial.readthedocs.org/en/latest/clip.html
    sf_ = shapefile.Reader(filename_shp)
    americas = [sf_.shapeRecords()[1], sf_.shapeRecords()[4]]
    vertices = []
    codes = []
    for shape_rec in americas:
    #shape_rec = sf_.shapeRecords()[num] # 1 north ame 4 south
        lons,lats = zip(*shape_rec.shape.points)
        pts = np.array(my_map(lons, lats)).T
        prt = list(shape_rec.shape.parts) + [len(pts)]
        for i in range(len(prt) - 1):
            for j in range(prt[i], prt[i+1]):
                vertices.append((pts[j][0], pts[j][1]))
            codes += [Path.MOVETO]
            codes += [Path.LINETO] * (prt[i+1] - prt[i] -2)
            codes += [Path.CLOSEPOLY]
    clip = Path(vertices, codes)
    clip = PathPatch(clip, transform=ax.transData)

    return clip  

In [158]:
filename, column = "data/Data_Puccia_paCelia.xls", ["LF"]

In [159]:
lllon = -180.
lllat = -80.
urlon = 0.
urlat = 84.
display = MainDisplay(lllon, lllat, urlon, urlat, fs='data/continents/continent')
# load ancestry and location data
map_data = MapData(filename, column)
map_data.get_coordinates()
#map_data.get_ancestry_average_by_coordinates(column[0])

map_data.project_coordinates(display.anc_map)
xi, yi, zi, x, y, z = map_data.interpolate()

clips = process_shapefile('data/continents/continent', display.anc_map, display.ax)
#print shape_clip
display.draw(xi, yi, zi, x, y, z, map_data.coordinates, map_data.df[column].values.ravel(), clips)

plt.title("Weigth to Heigth ratio: AF/HF (anchura facial/altura facial) - {}".format(column[0]))
# Cranial height-length: HN x 100/ LN.
# Orbital index: HO x 100/AO.
# Nasal index: AR x 100/ HR.
#plt.savefig("native_{}.png".format(column[1]), format="png", dpi=300, transparent=True)
plt.show()